# 데이터 셋 
- AI Hub에서 다운로드한 데이터셋은 json 파일이므로 LLM 학습에 적합하지 않음
- dataset_exam에서 수행한 parquet로 저장된 파일로 실습함
- colab 환경에서 진행하여 허깅페이스에 업로드 완료

In [ ]:
!pip install datasets

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [3]:
import pandas as pd
from datasets import dataset_dict, load_dataset, Dataset, DatasetDict
from tqdm import tqdm
import random
import string
import re

In [ ]:
import os
current_dir = os.getcwd()
default_path = current_dir + "/ai_hub/"
data = [d for d in os.listdir(default_path) if d != '.DS_Store' and d != 'Save']
print(data)


In [7]:
new_df=pd.read_parquet(default_path + '/Save/일반상식문장생성데이터.parquet')

In [8]:
filtered_df = new_df

In [ ]:
new_df

In [ ]:
# parquet 파일을 텍스트 형식으로 변환
# 전처리 1
from typing import Text

# 구두점이 있는 문장 합치기
for i in range(len(filtered_df)):
# for i in range(1000):
    # 공백 제거
    filtered_df['text'][i] = filtered_df['text'][i].strip()

    try:
        # 마침표 다음 스페이스 -> 문장 하나
        if filtered_df['text'][i].endswith('. '):
            filtered_df['text'][i]=filtered_df['Text'][i][:-2]
        elif filtered_df['text'][i].endswith('.'):
            pass
        elif filtered_df['text'][i].endswith(' .'):
            print(i, '번째 문장')
            print(filtered_df['text'[i]])
            filtered_df['text'][i] = filtered_df['text'][i][:-2]+'.'

            print(filtered_df['text'][i])
        else:
            print(i, '번째 문장')
            print(filtered_df['text'][i])

            # 가장 끝에 공백 -> 문장이 완료된 상태이므로 구두점(.) 추가
            if filtered_df['text'][i].endswith(' '):
                filtered_df['text'][i] = filtered_df['text'][i][:-1]+'.'
            else:
                filtered_df['text'][i] = filtered_df['text'][i]+'.'

            print(filtered_df['text'][i])

    except Exception as e:
        print(e)
        print(i)
        print(filtered_df['text'][i])

        print('################################################')



In [11]:
# 전처리 2 - 특수문자 제거
remove_index = []
for i in range(len(filtered_df)):
    text = filtered_df['text'][i]
    if '�' in text:
        remove_index.append(i)
    elif '삭제.' in text[-5:]:
        remove_index.append(i)

In [12]:
# 행삭제 및 인덱스 정리
filtered_df.drop(remove_index, inplace=True)
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
filtered_df

In [14]:
# 데이터 전처리 3 - 결측지 제거 : text 필드에 null인 행 제거
filtered_df=filtered_df.dropna(subset=['text'])

In [15]:
# 위의 전처리에서 행이 사라졌으므로 인덱스 리셋
filtered_df=filtered_df.reset_index(drop=True)

In [16]:
def cal_element(text):
    return len(text.split('. '))

In [19]:
# len_elements 필드와 텍스트 필드를 맞추는 작업. 이 예제에서는 .으로 문장 하나를 만드므로 1로 나옴
filtered_df['len_elements'] = filtered_df['text'].apply(lambda x: len(x.split('. ')) if isinstance(x, str) else 1)

In [ ]:
filtered_df['text'][0]

In [21]:
# 인덱스 초기화
filtered_df=filtered_df.reset_index(drop=True)

# Corpus 필터링
- 구두점이 중간에 찍히는 경우 과정

In [24]:
filtered_df_1=filtered_df[(filtered_df['len_elements'] >= 2)&(filtered_df['len_elements'] < 15 )] # 2이상 15인 경우 카운팅
filtered_df_2=filtered_df[filtered_df['len_elements'] < 2] # 정상 문장 카운팅

In [25]:
filtered_df_1=filtered_df_1.reset_index(drop=True)

In [ ]:
filtered_df['len_elements'].value_counts()

In [ ]:
filtered_df_1

In [29]:
# 잘못된 문장들을 올바르게 고치는 작업
filtered_df_1['text'] = ['하나에 열중하니 아무래도, 다른 것에는 소원하게 되었다.', '검은색, 붉은색, 녹색의 다채로운 쌀과, 찹쌀 등을 생산하고 있다.', '이 화장품 회사는 예전부터 꾸준히 1위 자리를 지키고 있다.']

In [30]:
filtered_df_1['len_elements'] = 1

In [ ]:
filtered_df_1

In [32]:
filtered_df_2=filtered_df_2.reset_index(drop=True)

In [ ]:
filtered_df_2

In [34]:
# 수정된 filtered_df_1을 filtered_df_2와 합침
filtered_df_2 = pd.concat([filtered_df_2, filtered_df_1], axis=0)

In [35]:
# 인덱스 초기화
filtered_df_2=filtered_df_2.reset_index(drop=True)

# 문장 추론 테스트
- 데이터가 잘 나오는지 확인하는 과정

In [37]:
# order inference를 진행하는 함수
def make_text_allign_data(text):
    if not text.endswith('.') or text.endswit('. '):
        text += '.'

    if text.endswith('.'):
        text+=' '
    
    my_list = text.split('. ')
    my_list_with_dots = [element + '.' for element in my_list]

    if my_list_with_dots[-1] == '.':
        my_list_with_dots=my_list_with_dots[:-1]

    random.shuffle(my_list_with_dots)

    if len(my_list_with_dots) == 1:
        raise Exception("allign 리스트 기이가 1입니다.", text)
    

    tmp_input='당신은 인공지능 비서입니다. 주어진 문장 리스트를 모두 활용하여 가장 정확한 단락을 생성하세요.'
    tmp_inst=f'이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.\n#문장 리스트: {my_list_with_dots}'

    if text.endswith(' '):
        tmp_out=text[:-1]
    else:
        tmp_out=text

    return tmp_input, tmp_inst, tmp_out


In [38]:
# 문장이 주어졌을때 마지막 문장 예측하는 함수
def make_completion_data(text):
  if not (text.endswith('.') or text.endswith('. ')):
    text += '.'

  if text.endswith('. '):
    text=text
  else:
    text = text +' '

  my_list_with_dots = text.split('. ')

  if my_list_with_dots[-1] == '':
    my_list_with_dots=my_list_with_dots[:-1]

    if len(my_list_with_dots) ==1:
      raise Exception("Completions 텍스트를 나눈 리스트의 길이가 1입니다.", text)

    last_sentence = my_list_with_dots.pop()
    last_sentence += '.'

  remaining_paragraph=''

  for i in range(len(my_list_with_dots)):
    if i!=len(my_list_with_dots)-1:
                 remaining_paragraph += my_list_with_dots[i]+'. '
    else:
      remaining_paragraph += my_list_with_dots[i]+'.' # 문장을 기억하고 있도록 설정

  tmp_input='당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.'
  tmp_instruct=f'다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞게 완성하세요. 문장은 이전 내용과 논리적으로 연결되어야 합니다.\n#텍스트: {remaining_paragraph}'
  tmp_output=last_sentence

  return tmp_input, tmp_instruct, tmp_output

In [39]:
# 괄호 안에 문장이 들어도록
def make_text_mask_data(text):
  if not (text.endswith('.') or text.endswith('. ')):
    text += '.'

  if text.endswith('. '):
    text=text[:-1]
  else:
    text = text

  words = re.findall(r'[가-힣]{2,}', text)
  random_word = random.choice(words)

  masked_text = text.replace(random_word, '<MASK>')

  tmp_input='당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.'
  tmp_instruct=f'이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를 예측해야 합니다. <MASK>가 위치한 문장의 전체 문맥을 분석하여, 문장의 나머지 내용과 일관되게 <MASK>에 들어갈 가장 적합한 단어를 답하세요.\n#텍스트: {masked_text}'
  tmp_output=random_word

  return tmp_input, tmp_instruct, tmp_output

In [83]:
def make_word_align(text):
  word_lst=[]
  for word in text.split(' '):
    out = re.sub(r"[^\w\s]", "", word)
    word_lst.append(out)

  #중복 제거하기 위해 set으로 만듬
  word_lst = set(word_lst)

  #다시 리스트 형식으로 돌림
  word_lst=list(word_lst)

  #랜덤하게 재배열
  random.shuffle(word_lst)

  tmp_input='당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.'
  tmp_instruct=f'이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들이 담긴 리스트가 제공됩니다. 이 리스트의 단어를 모두 활용하여 가장 문맥상 적절한 문장을 생성하세요.\n#단어리스트: {word_lst}'
  tmp_output=text

  return tmp_input, tmp_instruct, tmp_output

In [ ]:
# 데이터 생성 파이프라인
from os import TMP_MAX
input_lst=[]
output_lst=[]
inst_lst=[]
id_lst=[]

for i in tqdm(range(len(filtered_df_2))):
  try:
    text=filtered_df_2['text'][i]

    thred=random.random()

    if thred < 0.56:
      TMP_MAXtmp_id='word_align_aihub'
    else:
      # tmp_input, tmp_instruct, tmp_output = make_text_mask_data(text)
      tmp_input, tmp_instruct, tmp_output = make_completion_data(text)
      tmp_id='pre_mask_aihub'

    input_lst.append(tmp_input)
    inst_lst.append(tmp_instruct)
    output_lst.append(tmp_output)
    id_lst.append(tmp_id)

  except Exception as e:
    print(f"{i}번째 행")
    print(e)
    print(filtered_df_2['text'][i])
    print("---------------------------------------------")

In [ ]:
len(filtered_df_2)

In [71]:
hub_df_2=pd.DataFrame({'input':input_lst,'instruction':inst_lst,'output':output_lst})

In [72]:
hub_df = hub_df_2.copy()

In [73]:
hub_df=hub_df.reset_index(drop=True)

In [ ]:
hub_df

In [86]:
hub_df.to_parquet(default_path+'Save/일반상식문장생성데이터2.parquet', engine = 'pyarrow', index = False)

In [ ]:
! pip install datasets
! pip install huggingface_hub

In [ ]:
from huggingface_hub import login
from datasets import Dataset
login()

In [87]:
# 허깅페이스에 데이터 셋을 올리려면 판다스 형태로 변환해야함.
dataset = Dataset.from_pandas(hub_df)

In [ ]:
dataset

In [ ]:
dataset.push_to_hub('wonik-hi/korea_common_sence')